In [ ]:
# pip install together pandas matplotlib seaborn openpyxl python-docx PyMuPDF pillow pytesseract

In [41]:
import together
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import fitz 
from dotenv import load_dotenv
import pytesseract
from PIL import Image
import os

In [ ]:
load_dotenv()

In [45]:
def parse_file(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        return pd.read_excel(file_path)
    elif file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            return f.read()
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    elif file_path.endswith('.pdf'):
        doc = fitz.open(file_path)
        return "\n".join([page.get_text() for page in doc])
    elif file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        img = Image.open(file_path)
        return pytesseract.image_to_string(img)
    else:
        return "Unsupported file type"


In [ ]:
def ask_llama(prompt: str):
    response = together.Complete.create(
        model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
        prompt=prompt,
        max_tokens=1024,
        temperature=0.7
    )
    print("API response:", response)  
    return response['output']['choices'][0]['text'].strip()

In [47]:
def ask_llama(prompt: str):
    response = together.Complete.create(
        model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
        prompt=prompt,
        max_tokens=1024,
        temperature=0.7
    )
    if 'output' in response:
        return response['output']['choices'][0]['text'].strip()
    else:
        print("API Error:", response)
        return "API Error: " + str(response)

In [ ]:
def prompt_structured(df, question):
    sample = df.head(5).to_string()
    return f"""
You are a data analyst. Here's a sample of the dataset:

{sample}

{question}
Provide detailed analysis.
"""


In [ ]:
def prompt_unstructured(text, question):
    return f"""
You are an intelligent document analyst. Here's a portion of the document:

{text[:1000]}

Question:
{question}

"""


In [50]:
def basic_summary(df):
    print("Summary:")
    print(df.describe(include='all'))
    print("\nMissing Values:\n", df.isnull().sum())

def plot_column_counts(df, column):
    df[column].value_counts().plot(kind='bar', title=f'Distribution of {column}')
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)
    plt.show()

def plot_numeric_distribution(df, column):
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()


UPLOADING FILE

In [ ]:

file_path = "student_data.csv" 

parsed_data = parse_file(file_path)

if isinstance(parsed_data, pd.DataFrame):
    print(parsed_data.head())
    basic_summary(parsed_data)
    user_question = input("Question : ")
    final_prompt = prompt_structured(parsed_data, user_question)
else:
    print(parsed_data[:1000])
    user_question = input("Question : ")
    final_prompt = prompt_unstructured(parsed_data, user_question)


response = ask_llama(final_prompt)
print("\nAgent Response:\n")
print(response)


   Order ID Customer Name     Product         Category  Quantity  Price  \
0      1001    John Smith  Smartphone      Electronics         1    699   
1      1002   Alice Brown      Laptop      Electronics         1    999   
2      1003     Bob Davis     T-shirt          Apparel         3     25   
3      1004    Eve Turner     Blender  Home Appliances         1     45   
4      1005     Raj Patel       Shoes         Footwear         2     50   

   Total  Order Date Country  
0    699  2023-01-15     USA  
1    999  2023-01-16      UK  
2     75  2023-01-17   India  
3     45  2023-01-18  Canada  
4    100  2023-01-19   India  
Summary:
          Order ID Customer Name     Product     Category   Quantity  \
count     10.00000            10          10           10  10.000000   
unique         NaN            10          10            4        NaN   
top            NaN    John Smith  Smartphone  Electronics        NaN   
freq           NaN             1           1            4        N

/tmp/ipykernel_62197/3418193526.py:2: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/home/kapil/Downloads/VSCODE/venv/lib/python3.12/site-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


RateLimitError: Error code: 429 - {"message": "You have reached the rate limit specific to this model meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8. The maximum rate limit for this model is 0.6 queries and 180000000 tokens per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)", "type_": "model_rate_limit"}